In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from members import *

['Авраменко', 'Бубнова', 'Виноградова', 'Герасимов', 'Дятлов', 'Еналеев', 'Лебедев', 'Малышев', 'Ткачева']
Количество участников: 9
Количество женщин: 3


In [3]:
import numpy as np
import pandas as pd
import datetime

from collections import defaultdict
from collections import Counter

# Задача перечислить и распределить ресурсы

In [27]:
NBreakfast = 7
NDinner = 6
NLunch = 6
NSnack = 6

WomanFoodMaxWeight = 2000

In [52]:
Food = [# Продукты "ежедневного" потребления
         ('Соль'                      ,    7,    7,       Авраменко),
         ('Сахар'                     ,   60,    7,       ([Еналеев, Дятлов], [3.5, 3.5])),
         ('Чай (зеленый / черный)'    ,    7,    7,       Ткачева),
         ('Лимоны в сахаре'           ,   20,    7,       Дятлов),
         ('Перец'                     ,    1,    6,       Дятлов),
         ('Хмели-сунели'              ,    1,    6,       Дятлов),
         ('Аджика'                    ,    3,    6,       Ткачева),

        # Завтрак
        
         ('Геркулес Традиционный (з)' ,   40,    2,       Герасимов),
         ('Рис (завтрак)'             ,   50,    2,       Герасимов),
         ('Гречка (завтрак)'          ,   50,    2,       Герасимов),
         ('Пшено (завтрак)'           ,   50,    1,       Герасимов),
         ('Сухое молоко (завтрак)'    ,   25,    7,       Авраменко),
         ('Колбаса (завтрак)'         ,   40,    2,       Герасимов),
         ('Грудинка (завтрак)'        ,   40,    3,       Герасимов),
         ('Сыр (завтрак)'             ,   40,    2,       Герасимов), 
         ('Изюм (завтрак)'            ,   5,     7,       Авраменко),
         ('Чернослив (завтрак)'       ,   20,    1,       Ткачева),
         ('Cухое молоко в кофе (з)'   ,   2,     7,       Авраменко),
         ('Кофе'                      ,   2,     7,       Ткачева),
     
        # Перекус
     
         ('Колбаса (перекус)'         ,   40,    3,       Авраменко),
         ('Шоколад (перекус)'         ,   25,    6,       Виноградова),
         ('Орехи (перекус)'           ,   20,    6,       Авраменко),
         ('Сухари (перекус)'          ,   40,    6,       ([Виноградова, Малышев, Авраменко], [2, 2, 2])), 
         ('Курага (перекус)'          ,   10,    6,       Ткачева),
         ('Чернослив (перекус)'       ,   10,    6,       Ткачева),
     
         # Обед
         
         ('Чеснок'                    ,    4,    6,       Герасимов),
         ('Супы'                      ,    35,   6,       Лебедев),
     
         # Ужин
     
         ('Рис (ужин)'                ,   80,    1,       Малышев),
         ('Гречка (ужин)'             ,   80,    2,       Малышев),
         ('Чечевица (ужин)'           ,   80,    1,       Малышев),
         ('Спагетти (ужин)'           ,   90,    2,       Малышев),
         ('Пеммикан (ужин)'           ,   81,    6,       Лебедев),
         ('Лук'                       ,   10,    6,       Малышев),
         ('Сладости (ужин)'           ,   40,    6,       Бубнова),
         ('Спирт (ужин)'              ,   15,    6,       Еналеев),
        
        # Бензин
        
         ('Бензин'                    ,   100,   7,       ([Еналеев, Дятлов], [3.9, 3.1]))
]


names_counter = Counter()
total_weight = 0.0

for m in Members.values():
    m.food = []

tbl = []    

for row in Food:
    one_portion_weight = float(row[1])
    portion_weight = one_portion_weight * NMembers
    times = float(row[2])
    weight = portion_weight * times
        
    total_weight += weight

    assert len(row) == 3 or len(row) == 4
    names_counter[row[0]] += 1
    assert names_counter[row[0]] <= 1

    
    
    if len(row) == 3:
        tbl.append({'name': row[0], 
                'portion': portion_weight,
                'times': times,
                'weight': weight})

    else:
        if isinstance(row[3], Member):
            member = row[3]
            assert member.name in MembersList
            member.food.append({'kind' : 'food', 'packing_weight': portion_weight, 
                                'weight' : weight, 'name' : row[0]})
            
            tbl.append({'name': row[0], 
                'portion': portion_weight,
                'times': times,
                'weight': weight, 'Кто несет': member.name})
        else:
            members, ks = row[3]
            assert np.sum(ks) - times < 1e-9, print(np.sum(ks), times)
            assert len(members) == len(ks)
            assert len(members) > 1
            
            for member, k in zip(members, ks):
                assert member.name in MembersList
                member.food.append({'kind' : 'food', 'packing_weight': portion_weight, 
                                    'weight' : portion_weight * k, 'name' : row[0]})
                
                tbl.append({'name': row[0], 
                            'portion': portion_weight,
                            'times': k,
                            'weight': portion_weight * k, 'Кто несет': member.name})
            
            if np.sum(ks) < times - 1e-9:
                # Есть нераспределенное снаряжение
                tbl.append({'name': row[0], 
                            'portion': portion_weight,
                            'times': times - np.sum(ks),
                            'weight': portion_weight * (times - np.sum(ks))})

            
print("Общий вес: {}".format(total_weight))

avg = (total_weight - WomanFoodMaxWeight * NWomans) / (NMembers - NWomans)
print("Средний вес на мужчину: {}".format(avg))
print("Средний вес на женщину: {}".format(WomanFoodMaxWeight))
print("Вес на человека в день: {}\n".format(total_weight / NMembers / ((NBreakfast + NDinner + NSnack + NLunch) / 4)))

cur_weight = 0.0
for name, member in sorted(Members.items()):
    w = np.sum([r['weight'] for r in member.food])
    cur_weight += w
    if name in Womans:
        d = WomanFoodMaxWeight - w
    else:
        d = avg - w
    
    print("{}: {}".format(name, d))
    
print("\nОстаток: {}".format(total_weight - cur_weight))

table = pd.DataFrame(data=tbl)
table

Общий вес: 41544.0
Средний вес на мужчину: 5924.0
Средний вес на женщину: 2000
Вес на человека в день: 738.56

Авраменко: 587.0
Бубнова: -160.0
Виноградова: -70.0
Герасимов: 218.0
Дятлов: -124.0
Еналеев: -286.0
Лебедев: -340.0
Малышев: 164.0
Ткачева: 11.0

Остаток: 0.0


,name,portion,times,weight,Кто несет
0,Соль,63.0,7.0,441.0,Авраменко
1,Сахар,540.0,3.5,1890.0,Еналеев
2,Сахар,540.0,3.5,1890.0,Дятлов
3,Чай (зеленый / черный),63.0,7.0,441.0,Ткачева
4,Лимоны в сахаре,180.0,7.0,1260.0,Дятлов
5,Перец,9.0,6.0,54.0,Дятлов
6,Хмели-сунели,9.0,6.0,54.0,Дятлов
7,Аджика,27.0,6.0,162.0,Ткачева
8,Геркулес Традиционный (з),360.0,2.0,720.0,Герасимов
9,Рис (завтрак),450.0,2.0,900.0,Герасимов


### В 1.5л бутылке умещаятся 1150г бензина

In [10]:
for a, b in table.groupby(['Кто несет']):
        print("Участник: {}, несет: {}".format(a, b['weight'].sum()))
        print(str(b))
        print("\n\n")

with open("raskladka_wout_alexey.txt", 'w') as f:
    f.write("Дата создания: {}\n\n".format(datetime.datetime.now()))
    for a, b in table.groupby(['Кто несет']):
        f.write("Участник: {}, несет: {}\n".format(a, b['weight'].sum()))
        f.write(str(b))
        f.write("\n\n")
        
    f.write("\nСводная таблица")
    f.write(str(table))

Участник: Батырова, несет: 3780.0
        name  portion  times  weight Кто несет
24      Соль     49.0    9.0   441.0  Батырова
26     Сахар    490.0    2.7  1323.0  Батырова
35  Шиповник    224.0    9.0  2016.0  Батырова



Участник: Василец, несет: 5600.0
                 name  portion  times  weight Кто несет
5    Колбаса(завтрак)    385.0    3.0  1155.0   Василец
6   Грудинка(завтрак)    385.0    3.0  1155.0   Василец
7        Сыр(завтрак)    385.0    2.0   770.0   Василец
15        Спирт(ужин)    105.0    8.0   840.0   Василец
21   конфеты(перекус)    280.0    6.0  1680.0   Василец



Участник: Лебедев, несет: 6384.0
                name  portion  times  weight Кто несет
12    Сладости(ужин)    420.0    4.0  1680.0   Лебедев
14      Лимоны(ужин)    105.0    8.0   840.0   Лебедев
20  конфеты(перекус)    280.0    3.0   840.0   Лебедев
25             Сахар    490.0    2.7  1323.0   Лебедев
29               Чай     49.0    9.0   441.0   Лебедев
36  Сухофрукты+орехи    140.0    9.0  12

# Денежные затраты

## Сумма в кассе после покупки билетов 5097руб и оплаты палатки 1000руб

In [114]:
Cash = {Пименов: 5100 + 1000, 
        Василец: 5100, 
        Яничкин: 6754 + 1000, 
        Батырова: 5100 + 1000,
        Лидский:  5100 + 1000, 
        Начевский: 5100 + 1000}

Cash = dict([(k, v - 5097 - 1000) for k, v in Cash.items()])
Cash[Лебедев] = 0.0
Cash = np.array([Cash[k] for k in Members.values()])
print("\nОстаток в кассе на момент начала похода...")
for k, s in zip(Members.values(), Cash):
    print(k.name, s)



Остаток в кассе на момент начала похода...
Лебедев 0.0
Лидский 3.0
Начевский 3.0
Яничкин 1657.0
Василец -997.0
Батырова 3.0
Пименов 3.0


## Общественные траты

In [115]:
Expences = {Пименов: 1408,
            Василец: 2794,
            Лебедев: {'Конфеты' : 643 - 17 - 76,
                      'Сладости(ужин)' : 780,
                      'Сухофрукты+орехи' : 1400,
                      'Чай' : 1250 / 2, 
                      'Лимоны' : 125 * 0.6,
                      'Тушенка' : 3600,    # 225 * 16
                      'батарейки для GPS' : 81 * 4,
                      'машина в Лувеньгу' : 1500, 
                      'машина в Кандалакшу' : 2000, 
                      'GPS треккер'       : 250 * 8, 
                      'Батарейки к GPS треккеру' : 400}, 
            Яничкин: {'Оргстекло' : 100, 
                      'Грудинка'  : 500,
                      'Сало'      : 700,
                      'Сыр'       : 177,
                      'Сахар'     :  45}, 
            Батырова: {'сахар+шиповник+компот+соль' : 720, 
                       'оттяжки для палатки'        : 595}, 
            Лидский:  1000, 
            Начевский:  {'оттяжки для тента'   :  225,
                         'аптека'              :  610,
                         'продукты'            : 1210}
            }

stats = np.array([Expences[m] if not isinstance(Expences[m], dict) else np.sum(list(Expences[m].values())) for m in Members.values()])
print("Траты участников...")
for k, s in zip(Members.values(), stats):
    print(k.name, s)

avg = stats.mean()
print("\nСредний расход: {}".format(avg))

balance = stats - avg
print("\nБаланс...")
for k, s in zip(Members.values(), balance):
    print(k.name, s)



Траты участников...
Лебедев 13254.0
Лидский 1000.0
Начевский 2045.0
Яничкин 1522.0
Василец 2794.0
Батырова 1315.0
Пименов 1408.0

Средний расход: 3334.0

Баланс...
Лебедев 9920.0
Лидский -2334.0
Начевский -1289.0
Яничкин -1812.0
Василец -540.0
Батырова -2019.0
Пименов -1926.0


## Теперь учтем взносы каждого участника перед походом

In [122]:
for k, s in zip(Members.values(), balance + Cash):
    print(k.name, s)

Лебедев 9920.0
Лидский -2331.0
Начевский -1286.0
Яничкин -155.0
Василец -1537.0
Батырова -2016.0
Пименов -1923.0


In [123]:
1286 + 2016 - 2300*2

-1298

In [124]:
1923 + 1537

3460